Analysons les données

In [48]:
%pip install xgboost

In [49]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.3.1
Uninstalling scikit-learn-1.3.1:
  Successfully uninstalled scikit-learn-1.3.1


You can safely remove it manually.


   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.0 MB 10.1 MB/s eta 0:00:01
   -------------- ------------------------- 3.9/11.0 MB 9.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.3/11.0 MB 10.4 MB/s eta 0:00:01
   ------------------------------ --------- 8.4/11.0 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------  10.7/11.0 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 10.4 MB/s eta 0:00:00


Les imports


In [50]:
import pandas as pd
import numpy as np
from numpy import random
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score,f1_score
from sklearn.model_selection import train_test_split

In [51]:
data=pd.read_csv('train.csv',sep=',')

In [52]:
data

,ID,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,196348,0,Cash loans,F,N,Y,2,225000.0,785398.5,33403.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,147976,0,Cash loans,M,N,Y,0,90000.0,592560.0,35937.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,52662,0,Revolving loans,F,N,Y,1,126000.0,202500.0,10125.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
3,101577,0,Cash loans,F,N,Y,0,157500.0,254700.0,18531.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,173078,0,Cash loans,F,Y,N,0,540000.0,1252363.5,47830.5,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,Cash loans,F,N,Y,0,121500.0,675000.0,32602.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
215254,103694,0,Cash loans,F,N,Y,0,202500.0,1503000.0,41463.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
215255,131932,0,Cash loans,M,Y,Y,0,450000.0,1125000.0,33025.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,3.0
215256,146867,0,Revolving loans,M,Y,Y,1,180000.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [53]:
num_nan=data.isna().sum()
print(num_nan)

ID                                0
TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     29072
AMT_REQ_CREDIT_BUREAU_WEEK    29072
AMT_REQ_CREDIT_BUREAU_MON     29072
AMT_REQ_CREDIT_BUREAU_QRT     29072
AMT_REQ_CREDIT_BUREAU_YEAR    29072
Length: 122, dtype: int64


One possible idea that comes to mind is to replace nan values with random unique out of possible range values for numeric data and as for categorical data ,we create a new class called "unknown",because missing information is an information itself

extract numeric and categorical data

In [54]:
categorical=list(data.select_dtypes(exclude=["number","bool_"]).columns.values)
numeric=list(data.drop(columns=["TARGET"])._get_numeric_data().columns)

In [55]:
categorical

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE']

In [56]:
data[categorical]=data[categorical].fillna("unknown")

In [57]:

for col in numeric:
    min=float(np.abs(data[col].min()))
    alea=random.randint(100)*-1+min
    data[col]=data[col].fillna(alea)


In [58]:
np.unique(data.isna().sum())

array([0], dtype=int64)

As first step ,before deciding with which model will carry on , we need to study the nature of relationship between data

In [59]:
cols=list(data.drop(columns=["TARGET"]).columns.values)
cols

['ID',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_

ENCODING

In [60]:
features_encoded=pd.get_dummies(data,columns=categorical)
features_encoded

,ID,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,196348,0,2,225000.0,785398.5,33403.5,702000.0,0.046220,-13629,-514,...,False,False,False,False,False,False,True,False,False,True
1,147976,0,0,90000.0,592560.0,35937.0,450000.0,0.025164,-11983,-1863,...,False,False,False,True,False,False,False,True,False,False
2,52662,0,1,126000.0,202500.0,10125.0,202500.0,0.025164,-15182,-2033,...,False,False,False,False,False,False,True,False,False,True
3,101577,0,0,157500.0,254700.0,18531.0,225000.0,0.006207,-13214,-232,...,False,False,False,True,False,False,False,True,False,False
4,173078,0,0,540000.0,1252363.5,47830.5,1152000.0,0.072508,-19745,-713,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,0,121500.0,675000.0,32602.5,675000.0,0.006305,-14481,-1124,...,False,False,False,False,False,False,True,False,False,True
215254,103694,0,0,202500.0,1503000.0,41463.0,1503000.0,0.004960,-18097,-1557,...,False,False,False,False,False,False,True,False,False,True
215255,131932,0,0,450000.0,1125000.0,33025.5,1125000.0,0.014520,-14309,-5929,...,False,False,False,True,False,False,False,True,False,False
215256,146867,0,1,180000.0,270000.0,13500.0,270000.0,0.010006,-12706,-1868,...,False,False,False,False,True,False,False,True,False,False


In [61]:
#features_encoded=features_encoded.drop(columns=['CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Unknown'])

In [62]:
correlation_matrix = features_encoded.corr()
# Sélectionnez seulement la colonne de la variable target pour voir les corrélations avec toutes les autres variables
target_correlation = correlation_matrix['TARGET']
print(target_correlation)

ID                            -0.002793
TARGET                         1.000000
CNT_CHILDREN                   0.020044
AMT_INCOME_TOTAL              -0.001546
AMT_CREDIT                    -0.030230
                                 ...   
WALLSMATERIAL_MODE_Wooden      0.007796
WALLSMATERIAL_MODE_unknown     0.039865
EMERGENCYSTATE_MODE_No        -0.042609
EMERGENCYSTATE_MODE_Yes        0.005508
EMERGENCYSTATE_MODE_unknown    0.041690
Name: TARGET, Length: 252, dtype: float64


with these weak correlation ,it is preferable to avoid models that assume linearity with the target variable

Une technique très populaire dans la prédiction de default est le boosting

In [63]:
#add validation data
scaler=StandardScaler()
X=scaler.fit_transform(features_encoded.drop(columns=["TARGET"]))
print(X)
y=data["TARGET"]
x_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.1,random_state=42)


[[ 1.42774421  2.18879236  0.20436486 ... -1.03770792 -0.08656396
   1.05329067]
 [ 0.64930381 -0.57671339 -0.28733454 ...  0.9636623  -0.08656396
  -0.94940554]
 [-0.8845643   0.80603949 -0.1562147  ... -1.03770792 -0.08656396
   1.05329067]
 ...
 [ 0.3911111  -0.57671339  1.02386384 ...  0.9636623  -0.08656396
  -0.94940554]
 [ 0.63145691  0.80603949  0.04046506 ...  0.9636623  -0.08656396
  -0.94940554]
 [ 0.23060162 -0.57671339 -0.04148484 ... -1.03770792 -0.08656396
   1.05329067]]


In [64]:
model=xgb.XGBClassifier(objective='binary:logistic',eval_metric='logloss',use_label_encoder=False,early_stopping_rounds=10)
model.fit(x_train,y_train,eval_set=[(x_val,y_val)])

[0]	validation_0-logloss:0.28449
[1]	validation_0-logloss:0.27289
[2]	validation_0-logloss:0.26548
[3]	validation_0-logloss:0.25968
[4]	validation_0-logloss:0.25621


c:\Users\asmak\anaconda3\envs\python310\lib\site-packages\xgboost\core.py:158: UserWarning: [17:38:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[5]	validation_0-logloss:0.25361
[6]	validation_0-logloss:0.25212
[7]	validation_0-logloss:0.25099
[8]	validation_0-logloss:0.25009
[9]	validation_0-logloss:0.24938
[10]	validation_0-logloss:0.24870
[11]	validation_0-logloss:0.24795
[12]	validation_0-logloss:0.24762
[13]	validation_0-logloss:0.24746
[14]	validation_0-logloss:0.24718
[15]	validation_0-logloss:0.24701
[16]	validation_0-logloss:0.24665
[17]	validation_0-logloss:0.24646
[18]	validation_0-logloss:0.24635
[19]	validation_0-logloss:0.24608
[20]	validation_0-logloss:0.24600
[21]	validation_0-logloss:0.24581
[22]	validation_0-logloss:0.24580
[23]	validation_0-logloss:0.24590
[24]	validation_0-logloss:0.24582
[25]	validation_0-logloss:0.24558
[26]	validation_0-logloss:0.24559
[27]	validation_0-logloss:0.24549
[28]	validation_0-logloss:0.24545
[29]	validation_0-logloss:0.24541
[30]	validation_0-logloss:0.24531
[31]	validation_0-logloss:0.24530
[32]	validation_0-logloss:0.24530
[33]	validation_0-logloss:0.24543
[34]	validation_0-l

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

test data

In [65]:
test=pd.read_csv('test.csv',sep=',')
test.isna().sum()

ID                                0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
FLAG_OWN_REALTY                   0
                              ...  
AMT_REQ_CREDIT_BUREAU_DAY     12447
AMT_REQ_CREDIT_BUREAU_WEEK    12447
AMT_REQ_CREDIT_BUREAU_MON     12447
AMT_REQ_CREDIT_BUREAU_QRT     12447
AMT_REQ_CREDIT_BUREAU_YEAR    12447
Length: 121, dtype: int64

In [67]:
data

,ID,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,196348,0,Cash loans,F,N,Y,2,225000.0,785398.5,33403.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
1,147976,0,Cash loans,M,N,Y,0,90000.0,592560.0,35937.0,...,0,0,0,0,-34.0,-67.0,-40.0,-37.0,-90.0,-44.0
2,52662,0,Revolving loans,F,N,Y,1,126000.0,202500.0,10125.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0
3,101577,0,Cash loans,F,N,Y,0,157500.0,254700.0,18531.0,...,0,0,0,0,-34.0,-67.0,-40.0,-37.0,-90.0,-44.0
4,173078,0,Cash loans,F,Y,N,0,540000.0,1252363.5,47830.5,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,Cash loans,F,N,Y,0,121500.0,675000.0,32602.5,...,0,0,0,0,-34.0,-67.0,-40.0,-37.0,-90.0,-44.0
215254,103694,0,Cash loans,F,N,Y,0,202500.0,1503000.0,41463.0,...,0,0,0,0,-34.0,-67.0,-40.0,-37.0,-90.0,-44.0
215255,131932,0,Cash loans,M,Y,Y,0,450000.0,1125000.0,33025.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,3.0
215256,146867,0,Revolving loans,M,Y,Y,1,180000.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [66]:
test

,ID,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,37408,Cash loans,M,N,Y,0,90000.0,247500.0,19971.0,247500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
1,75132,Cash loans,M,N,N,1,121500.0,247275.0,19134.0,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,21513,Cash loans,F,N,Y,0,112500.0,679500.0,26946.0,679500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
3,58291,Revolving loans,F,N,Y,0,135000.0,405000.0,20250.0,405000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,39033,Cash loans,F,N,Y,0,121500.0,604413.0,21838.5,459000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92248,6265,Cash loans,M,N,N,0,225000.0,239850.0,25960.5,225000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
92249,54886,Revolving loans,M,N,Y,0,180000.0,247500.0,12375.0,247500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
92250,76820,Cash loans,F,Y,Y,0,135000.0,490536.0,31936.5,405000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
92251,860,Cash loans,M,Y,Y,0,270000.0,1247472.0,52983.0,1147500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0


In [68]:
features_encoded

,ID,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,196348,0,2,225000.0,785398.5,33403.5,702000.0,0.046220,-13629,-514,...,False,False,False,False,False,False,True,False,False,True
1,147976,0,0,90000.0,592560.0,35937.0,450000.0,0.025164,-11983,-1863,...,False,False,False,True,False,False,False,True,False,False
2,52662,0,1,126000.0,202500.0,10125.0,202500.0,0.025164,-15182,-2033,...,False,False,False,False,False,False,True,False,False,True
3,101577,0,0,157500.0,254700.0,18531.0,225000.0,0.006207,-13214,-232,...,False,False,False,True,False,False,False,True,False,False
4,173078,0,0,540000.0,1252363.5,47830.5,1152000.0,0.072508,-19745,-713,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215253,119879,0,0,121500.0,675000.0,32602.5,675000.0,0.006305,-14481,-1124,...,False,False,False,False,False,False,True,False,False,True
215254,103694,0,0,202500.0,1503000.0,41463.0,1503000.0,0.004960,-18097,-1557,...,False,False,False,False,False,False,True,False,False,True
215255,131932,0,0,450000.0,1125000.0,33025.5,1125000.0,0.014520,-14309,-5929,...,False,False,False,True,False,False,False,True,False,False
215256,146867,0,1,180000.0,270000.0,13500.0,270000.0,0.010006,-12706,-1868,...,False,False,False,False,True,False,False,True,False,False


In [69]:

categorical_test=list(test.select_dtypes(exclude=["number","bool_"]).columns.values)
numeric_test=list(test._get_numeric_data().columns)
test[categorical_test]=test[categorical_test].fillna("unknown")
for col in numeric_test:
    min=float(np.abs(test[col].min()))
    alea=random.randint(100)*-1+min
    test[col]=test[col].fillna(alea)
cols=list(test.columns.values)

features_encoded_test=pd.get_dummies(test,columns=categorical_test)




In [76]:
len(categorical)

16

In [77]:
len(categorical_test)

16

In [78]:
len(numeric)

105

In [79]:
len(numeric_test)

105

In [70]:
features_encoded_test

,ID,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_unknown,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_unknown
0,37408,0,90000.0,247500.0,19971.0,247500.0,0.028663,-16121,-582,-6226.0,...,False,False,False,False,True,False,False,True,False,False
1,75132,1,121500.0,247275.0,19134.0,225000.0,0.015221,-12586,-1133,-5319.0,...,False,False,False,False,False,False,True,False,False,True
2,21513,0,112500.0,679500.0,26946.0,679500.0,0.008625,-13473,-4466,-6354.0,...,False,False,False,False,False,False,True,False,False,True
3,58291,0,135000.0,405000.0,20250.0,405000.0,0.032561,-8468,-824,-8395.0,...,False,False,False,True,False,False,False,True,False,False
4,39033,0,121500.0,604413.0,21838.5,459000.0,0.035792,-22479,365243,-8837.0,...,False,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92248,6265,0,225000.0,239850.0,25960.5,225000.0,0.011703,-14120,-3751,-8112.0,...,False,False,False,False,True,False,False,True,False,False
92249,54886,0,180000.0,247500.0,12375.0,247500.0,0.019101,-13086,-2148,-3674.0,...,False,False,False,False,False,False,True,False,False,True
92250,76820,0,135000.0,490536.0,31936.5,405000.0,0.018029,-10403,-2403,-1124.0,...,False,False,False,False,False,False,True,False,False,True
92251,860,0,270000.0,1247472.0,52983.0,1147500.0,0.006852,-18571,-7045,-4269.0,...,False,False,False,False,False,False,True,False,False,True


In [71]:
l1=list(features_encoded.columns.values)
l2=list(features_encoded_test.columns.values)
diff=set(l1)-set(l2)
diff

{'CODE_GENDER_XNA', 'NAME_FAMILY_STATUS_Unknown', 'TARGET'}

In [73]:
test['CODE_GENDER']

0        M
1        M
2        F
3        F
4        F
        ..
92248    M
92249    M
92250    F
92251    M
92252    F
Name: CODE_GENDER, Length: 92253, dtype: object

In [82]:
features_encoded_test['CODE_GENDER_XNA']=False

In [83]:
features_encoded_test['CODE_GENDER_XNA']

0        False
1        False
2        False
3        False
4        False
         ...  
92248    False
92249    False
92250    False
92251    False
92252    False
Name: CODE_GENDER_XNA, Length: 92253, dtype: bool

In [85]:
features_encoded_test['NAME_FAMILY_STATUS_Unknown']=False

In [86]:
features_encoded_test['NAME_FAMILY_STATUS_Unknown']

0        False
1        False
2        False
3        False
4        False
         ...  
92248    False
92249    False
92250    False
92251    False
92252    False
Name: NAME_FAMILY_STATUS_Unknown, Length: 92253, dtype: bool

In [84]:
features_encoded[['NAME_FAMILY_STATUS_Unknown']]

,NAME_FAMILY_STATUS_Unknown
0,False
1,False
2,False
3,False
4,False
...,...
215253,False
215254,False
215255,False
215256,False


In [41]:
len(test.columns)

121

In [42]:
len(data.columns)

122

In [87]:

l1=list(features_encoded.columns.values)
l2=list(features_encoded_test.columns.values)
diff=set(l1)-set(l2)
diff



{'TARGET'}

In [88]:
scaler=StandardScaler()
X=scaler.fit_transform(features_encoded_test)

In [89]:
y_pred=model.predict(X)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [90]:
np.unique(y_pred)

array([0, 1])

In [91]:
predictions=list(y_pred)
id=list(test['ID'].values)

In [93]:
d={'ID':id,'TARGET':predictions}
submission=pd.DataFrame(data=d)



In [94]:
submission.to_csv('TEAMASMA.csv',index=None)